# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

## Implementação e experimentos com BM25

In [ ]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2

In [ ]:
import os
import datasets
import datetime
from typing import List
import pickle

from google.colab import userdata, drive

# from tqdm import tqdm
from pydantic import BaseModel, Field
# from typing import List, Optional, Literal

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings, OpenAIEmbeddings
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.retrievers import BaseRetriever
from rank_bm25 import BM25Okapi

from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### MedQA-USMLE-4-options dataset

In [ ]:
data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
sampled_question = data[42]

## MedQA

In [ ]:
# download MedQA data from https://drive.google.com/file/d/1ImYUSLk9JbgHXOemfvyiDiirluZHPeQw/view?usp=sharing
!gdown -q 1ImYUSLk9JbgHXOemfvyiDiirluZHPeQw
!unzip -q data_clean.zip -d /content/medQA

In [ ]:
def load_documents():
    '''
    Loads reference textbooks for MedQA as a list of Document objects.
    '''

    medqa_path = '/content/medQA/data_clean/textbooks/en/'
    loader = DirectoryLoader(medqa_path, glob='**/*.txt', loader_cls=TextLoader)

    # list of 'Document' files, with 'source'/file path as metadata
    documents = loader.load()

    return documents

In [ ]:
docs = load_documents()

In [ ]:
# splits list of documents into chunks and adds the source txt filename as metadata
chunk_size = 500
overlap_size = 50
# chunk embedding model
emb_model_name = 'sentence-transformers/all-MiniLM-L6-v2'

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap_size)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## BM25 Okapi from langchain

In [ ]:
split_documents = text_splitter.create_documents([doc.page_content for doc in docs], metadatas=[{'ref': doc.metadata['source'].split('/')[-1]} for doc in docs])
retriever_docs = BM25Retriever.from_documents(split_documents)

In [ ]:
#Saves retriever in a file
with open('/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega3/retriever/bm25_retriever.pkl', 'wb') as f:
    pickle.dump(retriever_docs, f)

## BM25 corpus

In [ ]:
def create_bm25_corpus(docs, text_spliter):
  '''
  Creates corpus to be used in BM25.
  Also, maps the metadata of the documents.
  '''

  #Splits documents into chunks
  split_documents = text_splitter.create_documents([doc.page_content for doc in docs], metadatas=[{'ref': doc.metadata['source'].split('/')[-1]} for doc in docs])
  #Transforms in a list with only the content
  corpus = [doc.page_content for doc in split_documents]
  #Maps the reference with each content chunk
  metadata_mapping = {doc.page_content: doc.metadata for doc in split_documents}
  return corpus, metadata_mapping

In [ ]:
bm25_corpus, metadata_mapping = create_bm25_corpus(docs, text_splitter)

## BM25 Okapi and Reciprocal Rank Fusion

In [ ]:
def load_vector_store(embed_model):
    # load saved faiss indexes
    faiss = FAISS.load_local(
        folder_path='/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega3/retriever/faiss__500_50__20241103_135850',
        embeddings=embed_model,
        allow_dangerous_deserialization=True
    )

    return faiss

In [ ]:
# vector store for retrieval
faiss = load_vector_store(embeddings)

In [ ]:
class VectorDbWithBM25:
  def __init__(self, bm25_corpus, vector_store, metadata_mapping):
    self.bm25_corpus = bm25_corpus
    self.vector_store = vector_store
    self.metadata_mapping = metadata_mapping

    #Tokenizes the BM25 corpus
    tokenized_corpus = [doc.split(' ') for doc in bm25_corpus]
    self.bm25 = BM25Okapi(bm25_corpus)

  def add_metadata(self, results):
    ''' Adds metadata to the results dictionary.
    '''
    results_with_metadata = {}
    for doc, scores in results.items():
      metadata = self.metadata_mapping.get(doc, {})
      results_with_metadata[doc] = {'score': scores, **metadata}

    return results_with_metadata

  @lru_cache(maxsize=128)
  def bm25_search(self, query, k=3):
    '''BM25 search.
    Returns top k documents with scores.
    '''
    tokenized_query = query.split(' ')
    doc_scores = self.bm25.get_scores(tokenized_query)
    docs_with_scores = dict(zip(self.bm25_corpus, doc_scores))
    sorted_docs_with_scores = sorted(docs_with_scores.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_docs_with_scores[:k])

  def vector_search(self, query, k=3):
    ''' Vector similarity search.
    Returns top k documents with scores.
    '''
    search_result = dict()
    docs_and_score = self.vector_store.similarity_search_with_relevance_scores(query=query, k=k)
    for doc, score in docs_and_score:
      search_result[doc.page_content] = score
    return search_result

  def combine_results(self, bm25_results, vector_results, epsilon=0.05):
    '''
    Combines the results of BM25 and vector search.
    '''
    def normalize_dict(input_dict):
      '''
      Normalizes the values of a dictionary.
      '''
      min_value = min(input_dict.values())
      max_value = max(input_dict.values())
      a = 0.05
      b = 1

      if max_value == min_value:
        return {k: b if max_value > 0.5 else a for k in input_dict.keys()}

      return {k: a + ((v - min_value) / (max_value - min_value)) * (b-a) for k, v in input_dict.items()}

    bm25_results_norm = normalize_dict(bm25_results) #Normalize BM25 scores
    vector_results_norm = normalize_dict(vector_results) #Normalize faiss scores

    #Combine dictionaries
    #Merges both dictionaries, if a document was retrieved for both algorithms, appends the scores
    combined_dict = {}
    for k, v in vector_results_norm.items():
      combined_dict.setdefault(k, []).append(v)

    for k, v in bm25_results_norm.items():
      combined_dict.setdefault(k, []).append(v)

    return combined_dict

  def search(self, query, k=3):
    ''' Execute both search algorithms, BM25 and FAISS similarity search and combine the results.
    '''
    with ThreadPoolExecutor(max_workers=2) as executor:
      bm25_future = executor.submit(self.bm25_search, query, k)
      vector_future = executor.submit(self.vector_search, query, k)

      bm25_results = bm25_future.result()
      vector_results = vector_future.result()

      if len(vector_results) == 0:
        return self.add_metadata(bm25_results)
      elif len(bm25_results) == 0:
        return self.add_metadata(vector_results)
      else:
        combined_results = self.combine_results(bm25_results, vector_results)
        sorted_combined_results = dict(sorted(combined_results.items(), key=lambda x: x[1], reverse=True))

        #Adds metadata to final dict
        results_with_metadata = self.add_metadata(sorted_combined_results)

        return results_with_metadata

  def reciprocal_rank_fusion(self,search_results, k=60, top_k=3):
    '''
    If the document has scores from both BM25 and FAISS
    (it is a list with scores from both search models), it calculates the Reciprocal Rank Fusion score
    Returns top k documents with scores
    '''
    fused_scores = {}

    for doc, data in search_results.items():
      scores = data['score']
      metadata = {key: value for key, value in data.items() if key != 'score'} #Extract metadata
      if len(scores) > 1:
          # For cases where both FAISS and BM25 scores are available
          rrf_score = 0
          for rank, _ in enumerate(sorted(scores, reverse=True)):
            rrf_score += 1 / (rank + 1)
            fused_scores[doc] = {'score': rrf_score, **metadata}
      else:
          # If only one score is available (e.g., only BM25)
          fused_scores[doc] = {'score': scores[0], **metadata}  # Use the first score as the fused score

    sorted_fused_scores =  dict(sorted(fused_scores.items(), key=lambda item: item[1]['score'], reverse=True))
    return dict(list(sorted_fused_scores.items())[:top_k])


In [ ]:
vector_bm25_search = VectorDbWithBM25(bm25_corpus, faiss, metadata_mapping)

### Custom Retriever


In [ ]:
class VectorDbWithBM25Retriever(BaseRetriever, BaseModel):
  """A retriever that contains the top k documents that contain the user query.

  This retriever only implements the sync method _get_relevant_documents.
  """
  bm25_corpus: List[str]
  faiss: object
  metadata_mapping: dict
  k: int
  top_k: int

  class Config:
        arbitrary_types_allowed = True

  def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
    """Sync implementations for retriever."""
    vector_bm25_search = VectorDbWithBM25(self.bm25_corpus, self.faiss, self.metadata_mapping)
    search_results = vector_bm25_search.search(query, k=self.k)
    matching_documents = vector_bm25_search.reciprocal_rank_fusion(search_results, top_k=self.top_k)

    document_list = []
    for content, data in matching_documents.items():
        metadata = {'ref': data['ref']}
        doc = Document(page_content=content, metadata=metadata)
        document_list.append(doc)

    return document_list

In [ ]:
retriever_vectorwbm25 = VectorDbWithBM25Retriever(bm25_corpus=bm25_corpus, faiss=faiss, metadata_mapping=metadata_mapping, k=6, top_k=3)

## Testing

In [ ]:
sampled_question['question']

'A one-day-old male is evaluated in the hospital nursery for bilious vomiting. The patient has urinated multiple times since birth but has not yet passed meconium. He was born at 38 weeks gestation to a 36-year-old gravida 3 via vaginal delivery. The pregnancy was complicated by gestational diabetes. The patient’s mother received routine prenatal care and declined first trimester screening. She subsequently underwent a quadruple screen at 15 weeks gestation that demonstrated decreased maternal serum alpha-fetoprotein (AFP), increased beta-hCG, decreased unconjugated estriol, and increased inhibin A. Her last ultrasound prior to onset of labor demonstrated an amniotic fluid index (AFI) of 28 cm. The patient’s two older siblings are both healthy. The patient’s temperature is 97.8°F (36.6°C), blood pressure is 58/37 mmHg, pulse is 166/min, and respirations are 38/min. On physical exam, the patient is in moderate distress. His abdomen is firm and distended with hypoactive bowel sounds.\n\n

In [ ]:
sampled_question['options']

{'A': 'Duodenal atresia',
 'B': 'Meconium ileus',
 'C': 'Necrotizing enterocolitis',
 'D': 'Pyloric stenosis'}

In [ ]:
retriever_docs.invoke('Duodenal atresia')

[Document(metadata={'ref': 'Obstentrics_Williams.txt'}, page_content='Dandy-Walker malformation 1/12,000 40 18/ 13/21, triploidy Clet lip/palate 1/1000 5-15 18,13 Cardiac defects 5-8/1000 10-30 21; 18; 13; 45/X; 22q 11.2 microdeletion\n\nDiaphragmatic hernia 1/3000-4000 5-15 18,l13,l21\n\nEsophageal atresia 1/4000 10 18,l21\n\nDuodenal atresia 1/10,000 30 21 Gastroschisis 112000-4000 No increase'),
 Document(metadata={'ref': 'First_Aid_Step1.txt'}, page_content='Intestinal atresia Presents with bilious vomiting and abdominal distension within first 1–2 days of life. Duodenal atresia—failure to recanalize. Abdominal x-ray A shows “double bubble” (dilated stomach, proximal duodenum). Associated with Down syndrome. Jejunal and ileal atresia—disruption of mesenteric vessels (typically SMA) \x8e ischemic necrosis of fetal intestine \x8e segmental resorption: bowel becomes discontinuous. X-ray shows dilated loops of small bowel with air-fluid levels.'),
 Document(metadata={'ref': 'First_Aid_

In [ ]:
retriever_vectorwbm25.invoke('Duodenal atresia')

[Document(metadata={'ref': 'First_Aid_Step2.txt'}, page_content='in the stomach and duodenum) proximal to the site of the atresia. Treatment: Surgical repair.'),
 Document(metadata={'ref': 'Obstentrics_Williams.txt'}, page_content='Duodenal atresia is found in approximately 1 in 10,000 births (Best, 2012; Dolk, 2010). It is characterized by the sonographic double-bubble sin, which represents distention of the stomach and the irst part of the duodenum (Fig. 10-38). This finding is usually not present before 22 to 24 weeks\' gestation. Demonstrating continuity between the stomach and proximal duodenum confirms that the second "bubble" is the proximal duodenum. Approximately 30 percent of afected fetuses have an associated'),
 Document(metadata={'ref': 'Surgery_Schwartz.txt'}, page_content='favor performing a duodeno-duodenostomy for children with duodenal web, although such an approach may lead to long-term complications associated with the creation of a blind section of duodenum between

### Testing llm

In [ ]:
model = 'gpt-4o-mini'
model_temp = 0.5

In [ ]:
llm = ChatOpenAI(
    model=model,
    temperature=model_temp,
)

In [ ]:
tool_search_bm25 = create_retriever_tool(
    retriever=retriever_docs,
    name='search_docs',
    description='Searchs the query in documents and returns the top-3 most relevant ones.',
)

In [ ]:
print(tool_search_bm25.invoke('meconium ileus'))

meconium ileus, the terminal ileum is filled with pellets of meconium. In patients with complicated meco-nium ileus, intraperitoneal calcifications form, producing an eggshell pattern on plain abdominal X-ray.Management. The treatment strategy depends on whether the patient has complicated or uncomplicated meconium ileus. Patients with uncomplicated meconium ileus can be Brunicardi_Ch39_p1705-p1758.indd   172612/02/19   11:26 AM 1727PEDIATRIC SURGERYCHAPTER 39Figure 39-17. Ladd procedure for

Diagnosis: Meconium ileus (obstruction of the ileum by meconium, the first stool produced by newborns) was confirmed by abdominal x-rays. About 98% of full-term newborns with meconium ileus have cystic fibrosis (CF). Diagnosis of CF was subsequently confirmed with a chloride sweat test.

Treatment: The ileus was successfully treated nonsurgically. For management of the CF, the family was referred to the CF center at the regional children’s hospital.

emesis. Abdominal radiographs show dilated loop

In [ ]:
tool_search_vectorwbm25 = create_retriever_tool(
    retriever=retriever_vectorwbm25,
    name='search_docs',
    description='Searchs the query in documents and returns the top-3 most relevant ones.',
)

In [ ]:
print(tool_search_vectorwbm25.invoke('meconium ileus'))

emesis. Abdominal radiographs show dilated loops of intestine. Because the enteric contents are so viscous, air-fluid levels do not form, even when obstruction is complete. Small bubbles of gas become entrapped in the inspissated meconium in the dis-tal ileum, where they produce a characteristic “ground glass” appearance.The diagnosis of meconium ileus is confirmed by a con-trast enema that typically demonstrates a microcolon. In patients with uncomplicated meconium ileus, the terminal ileum is

meconium ileus, the terminal ileum is filled with pellets of meconium. In patients with complicated meco-nium ileus, intraperitoneal calcifications form, producing an eggshell pattern on plain abdominal X-ray.Management. The treatment strategy depends on whether the patient has complicated or uncomplicated meconium ileus. Patients with uncomplicated meconium ileus can be Brunicardi_Ch39_p1705-p1758.indd   172612/02/19   11:26 AM 1727PEDIATRIC SURGERYCHAPTER 39Figure 39-17. Ladd procedure for

D

In [ ]:
# react prompt
system_prompt = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do. Break the problem down into subproblems and smaller steps and decide which action to take.
Action: the action to take, must be one of [{tool_names}].
Action Input: the input to the action
Observation: the output of the action.
... (this Thought/Action/Action Input/Observation can repeat any number of times)
Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
prompt_template = ChatPromptTemplate.from_template(system_prompt)

In [ ]:
agent = create_react_agent(
    llm=llm,
    tools=[tool_search_bm25],
    prompt=prompt_template
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[tool_search_bm25],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

In [ ]:
res = agent_executor.invoke({'question': sampled_question['question'], 'options': sampled_question['options']})



> Entering new AgentExecutor chain...
To determine the most likely etiology of this one-day-old male's presentation, we need to analyze the symptoms and medical history provided. The key points to consider are:

1. **Bilious vomiting**: This suggests an obstruction in the gastrointestinal tract.
2. **Not passed meconium**: In a newborn, failure to pass meconium within the first 24 hours can indicate a blockage.
3. **Firm, distended abdomen with hypoactive bowel sounds**: This further supports the idea of an obstruction.

Next, we will evaluate the potential options:
- **A: Duodenal atresia**: This condition can present with bilious vomiting and failure to pass meconium. It is often associated with Down syndrome and can involve a "double bubble" sign on X-ray.
- **B: Meconium ileus**: This is commonly seen in cystic fibrosis patients and also leads to failure to pass meconium and can cause bilious vomiting.
- **C: Necrotizing enterocolitis**: This condition typically presents in prema

In [ ]:
res

{'question': 'A one-day-old male is evaluated in the hospital nursery for bilious vomiting. The patient has urinated multiple times since birth but has not yet passed meconium. He was born at 38 weeks gestation to a 36-year-old gravida 3 via vaginal delivery. The pregnancy was complicated by gestational diabetes. The patient’s mother received routine prenatal care and declined first trimester screening. She subsequently underwent a quadruple screen at 15 weeks gestation that demonstrated decreased maternal serum alpha-fetoprotein (AFP), increased beta-hCG, decreased unconjugated estriol, and increased inhibin A. Her last ultrasound prior to onset of labor demonstrated an amniotic fluid index (AFI) of 28 cm. The patient’s two older siblings are both healthy. The patient’s temperature is 97.8°F (36.6°C), blood pressure is 58/37 mmHg, pulse is 166/min, and respirations are 38/min. On physical exam, the patient is in moderate distress. His abdomen is firm and distended with hypoactive bowe

In [ ]:
sampled_question['answer_idx']

'A'

In [ ]:
agent = create_react_agent(
    llm=llm,
    tools=[tool_search_vectorwbm25],
    prompt=prompt_template
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[tool_search_vectorwbm25],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

In [ ]:
res = agent_executor.invoke({'question': sampled_question['question'], 'options': sampled_question['options']})



> Entering new AgentExecutor chain...
To determine the most likely etiology of the one-day-old male's presentation, I need to analyze the clinical details provided. The key points include:

1. **Bilious vomiting**: Indicates that there is a blockage or obstruction in the gastrointestinal tract.
2. **Not passed meconium**: Suggests a potential obstruction or issue with the intestines, as meconium passage usually occurs within the first 24 hours of life.
3. **Physical exam findings**: The abdomen is firm and distended with hypoactive bowel sounds, which further supports the idea of an obstruction.

Next, I will evaluate the options provided:

- **A: Duodenal atresia**: Commonly presents with bilious vomiting and is associated with failure to pass meconium.
- **B: Meconium ileus**: Typically seen in cystic fibrosis patients, it can also cause bilious vomiting and failure to pass meconium.
- **C: Necrotizing enterocolitis**: More common in premature infants and usually presents with othe

In [ ]:
res

{'question': 'A one-day-old male is evaluated in the hospital nursery for bilious vomiting. The patient has urinated multiple times since birth but has not yet passed meconium. He was born at 38 weeks gestation to a 36-year-old gravida 3 via vaginal delivery. The pregnancy was complicated by gestational diabetes. The patient’s mother received routine prenatal care and declined first trimester screening. She subsequently underwent a quadruple screen at 15 weeks gestation that demonstrated decreased maternal serum alpha-fetoprotein (AFP), increased beta-hCG, decreased unconjugated estriol, and increased inhibin A. Her last ultrasound prior to onset of labor demonstrated an amniotic fluid index (AFI) of 28 cm. The patient’s two older siblings are both healthy. The patient’s temperature is 97.8°F (36.6°C), blood pressure is 58/37 mmHg, pulse is 166/min, and respirations are 38/min. On physical exam, the patient is in moderate distress. His abdomen is firm and distended with hypoactive bowe